In [1]:
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

In [2]:
os.listdir("./Dataset")

['finger_1', 'finger_2', 'finger_3', 'finger_4', 'finger_5']

In [3]:
def check_and_remove_corrupt_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            try:
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                img.verify()
            except (IOError, SyntaxError, Image.UnidentifiedImageError) as e:
                print(f"File rusak ditemukan dan dihapus: {img_path}")
                os.remove(img_path)

check_and_remove_corrupt_images('Dataset/')
print("Pemeriksaan dan penghapusan file rusak selesai.")

Pemeriksaan dan penghapusan file rusak selesai.


In [4]:
def threshold_dan_balik(image, batas):
    hasil = np.copy(image)
    hasil[hasil>=batas] = 255
    hasil[hasil<batas] = 0

    for i in range(hasil.shape[0]):
        for j in range(hasil.shape[1]):
            if(hasil[i][j]==0):
                hasil[i][j]=255
            else:
                hasil[i][j]=0
    return hasil

In [5]:
def convo_edge(image, kernel):
    img_height, img_width = image.shape
    kernel_height, kernel_width = kernel.shape
    pad_height = kernel_height //2
    pad_width = kernel_width //2
    padded_img =  np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')
    result = np.zeros([img_height, img_width])

    for i in range(img_height):
        for j in range(img_width):
            result[i, j] = np.sum(padded_img[i:i+kernel_height, j:j+kernel_width] * kernel)
    return result

Sx = np.array([[-1,0,1],
               [-2,0,2],
               [-1,0,1]])

Sy = np.array([[1,2,1],
               [0,0,0],
               [-1,-2,-1]])

def deteksi_tepi(image, kernelX, kernelY):
    hasilX = convo_edge(image, kernelX)
    hasilY = convo_edge(image, kernelY)

    M = np.sqrt((hasilX*hasilX)+(hasilY*hasilY))

    return M

In [6]:
kernel = np.array([[1,1,1],
                   [1,1,1],
                   [1,1,1]])
def erosi(image, kernel):
    height, width = image.shape
    k_height, k_width = kernel.shape
    center = k_height // 2
    hasil = np.zeros((height, width))
    for i in range(center, height-center):
        for j in range(center, width-center):
            cocok = True
            for k in range(k_height):
                for l in range(k_width):
                    if kernel[k, l] == 1 and image[i + k - center, j + l - center] == 0:
                        cocok = False
                        break
                if not cocok:
                    break
            if cocok:
                hasil[i, j] = 255
    return hasil

In [7]:
def dilasi(image,kernel):
    height, width = image.shape
    k_height, k_width = kernel.shape
    center = k_height//2
    hasil = np.zeros((height, width))
    for i in range(center, height-center):
        for j in range(center, width-center):
            if image[i,j] == 255:
                for k in range(k_height):
                    for l in range(k_width):
                        if kernel[k,l] == 1:
                            hasil[i+k-center,j+l-center] =255
            else:
                if hasil[i,j] !=255:
                    hasil[i,j] = 0 
    return hasil

In [8]:
def closing(dilasi, kernel):
    hasil = erosi(dilasi, kernel)
    return hasil

In [9]:
dataset_dir = "./Dataset"
preprocessing_dir = "./Preprocessing_RGB"

if not os.path.exists(preprocessing_dir): # kalo foldernya belum ada, bikin dulu
    os.makedirs(preprocessing_dir)

def preprocess_image(image_path, output_path):
    with Image.open(image_path) as img:
        img = img.convert('RGB')
        img = np.array(img)
        img = cv2.resize(img, (150, 150))
        cv2.imwrite(output_path, img) 

total_images = sum(len(files) for _, _, files in os.walk(dataset_dir) if files)

with tqdm(total=total_images, desc="Processing Images") as pbar:
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_dir):
            # Membuat folder kelas di dalam folder preprocessing
            new_class_dir = os.path.join(preprocessing_dir, class_name)
            os.makedirs(new_class_dir, exist_ok=True)
            
            # Iterasi melalui setiap gambar dalam folder kelas
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                new_image_path = os.path.join(new_class_dir, image_name)
                preprocess_image(image_path, new_image_path)
                pbar.update(1)

print("Preprocessing selesai!")

Processing Images: 100%|██████████| 2099/2099 [03:10<00:00, 11.00it/s]

Preprocessing selesai!
